In [ ]:
from kirby.basic_model import BasicModel
from kirby.run_params import RunParams
from kirby.data_manager import DataManager
from kirby.dataset_builder import DatasetBuilder
from transformers import GPT2Tokenizer
import spacy
import json
import torch

In [ ]:
rp = RunParams()
model = BasicModel(rp)
dm = DataManager(rp)
db = DatasetBuilder()

In [ ]:
ds = dm.load('train')

Using custom data configuration default-d64f335cc8a13d66
Reusing dataset text (/home/rob/.cache/huggingface/datasets/text/default-d64f335cc8a13d66/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
Loading cached processed dataset at /home/rob/.cache/huggingface/datasets/text/default-d64f335cc8a13d66/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5/cache-afe4d9c5e6fd685f.arrow


In [ ]:
x = ds[0]['text']
x

' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . '

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [ ]:
doc = nlp(x)
entities = nlp(x).ents
for e in entities:
    print(e)

Unrecorded Chronicles
Japanese
Valkyria Chronicles III
Japan
Sega
January 2011
Japan
third
first
the " Nameless "
the Second Europan War
Imperial
Calamaty Raven "


In [ ]:
e = entities[2]
e

Valkyria Chronicles III

In [ ]:
entity = db.db.get_entity_by_label(e.text)
entity

['Q5954954', 'Valkyria Chronicles III', '2011 video game']

In [ ]:
entity_dict = db.get_entity_associations(entity)
entity_dict

{'id': 'Q5954954',
 'description': '2011 video game',
 'instance of': 'video game',
 'game mode': 'single-player video game',
 'part of the series': 'Valkyria Chronicles',
 'publisher': 'Sega',
 'composer': 'Hitoshi Sakimoto',
 'distribution format': 'Universal Media Disc',
 'genre': 'tactical role-playing game',
 'platform': 'PlayStation Portable',
 'developer': 'Media.Vision',
 'country of origin': 'Japan',
 'follows': 'Valkyria Chronicles II',
 'followed by': 'Valkyria Chronicles 4'}

In [ ]:
type(entity_dict)

dict

In [ ]:
knowledge = list(entity_dict.items())
knowledge

[('id', 'Q5954954'),
 ('description', '2011 video game'),
 ('instance of', 'video game'),
 ('game mode', 'single-player video game'),
 ('part of the series', 'Valkyria Chronicles'),
 ('publisher', 'Sega'),
 ('composer', 'Hitoshi Sakimoto'),
 ('distribution format', 'Universal Media Disc'),
 ('genre', 'tactical role-playing game'),
 ('platform', 'PlayStation Portable'),
 ('developer', 'Media.Vision'),
 ('country of origin', 'Japan'),
 ('follows', 'Valkyria Chronicles II'),
 ('followed by', 'Valkyria Chronicles 4')]

In [ ]:
k = {entity[1]: {knowledge[1][0] : knowledge[1][1]}}
k

{'Valkyria Chronicles III': {'description': '2011 video game'}}

In [ ]:
i = x + json.dumps(k)
i

' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . {"Valkyria Chronicles III": {"description": "2011 video game"}}'

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(rp.model)

In [ ]:
a = ds[0]
a

{'text': ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . '}

In [ ]:
a['knowledge'] = json.dumps(k)
a

{'text': ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . ',
 'knowledge': '{"Valkyria Chronicles III": {"description": "2011 video game"}}'}

In [ ]:
t = dm.tokenize(a, tokenizer)

In [ ]:
t

{'input_ids': [2311, 73, 13090, 645, 569, 18354, 7496, 513, 1058, 791, 47398, 17740, 357, 4960, 1058, 10545, 230, 99, 161, 254, 112, 5641, 44444, 9202, 25084, 24440, 12675, 11839, 18, 837, 6578, 764, 569, 18354, 7496, 286, 262, 30193, 513, 1267, 837, 8811, 6412, 284, 355, 569, 18354, 7496, 17740, 6711, 2354, 2869, 837, 318, 257, 16106, 2597, 2488, 12, 31, 2712, 2008, 983, 4166, 416, 29490, 290, 6343, 13, 44206, 329, 262, 14047, 44685, 764, 28728, 287, 3269, 2813, 287, 2869, 837, 340, 318, 262, 2368, 983, 287, 262, 569, 18354, 7496, 2168, 764, 12645, 278, 262, 976, 21748, 286, 16106, 290, 1103, 2488, 12, 31, 640, 11327, 355, 663, 27677, 837, 262, 1621, 4539, 10730, 284, 262, 717, 983, 290, 5679, 262, 366, 17871, 5321, 366, 837, 257, 23634, 2422, 4326, 7351, 262, 3277, 286, 7096, 544, 1141, 262, 5498, 1898, 6839, 1810, 508, 1620, 3200, 2042, 4560, 290, 389, 46852, 1028, 262, 11773, 4326, 366, 2199, 321, 265, 88, 12552, 366, 764, 220], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
t['labels'] = t['input_ids']

In [ ]:
t['knowledge'] = tokenizer(a['knowledge'])
t

{'input_ids': [2311, 73, 13090, 645, 569, 18354, 7496, 513, 1058, 791, 47398, 17740, 357, 4960, 1058, 10545, 230, 99, 161, 254, 112, 5641, 44444, 9202, 25084, 24440, 12675, 11839, 18, 837, 6578, 764, 569, 18354, 7496, 286, 262, 30193, 513, 1267, 837, 8811, 6412, 284, 355, 569, 18354, 7496, 17740, 6711, 2354, 2869, 837, 318, 257, 16106, 2597, 2488, 12, 31, 2712, 2008, 983, 4166, 416, 29490, 290, 6343, 13, 44206, 329, 262, 14047, 44685, 764, 28728, 287, 3269, 2813, 287, 2869, 837, 340, 318, 262, 2368, 983, 287, 262, 569, 18354, 7496, 2168, 764, 12645, 278, 262, 976, 21748, 286, 16106, 290, 1103, 2488, 12, 31, 640, 11327, 355, 663, 27677, 837, 262, 1621, 4539, 10730, 284, 262, 717, 983, 290, 5679, 262, 366, 17871, 5321, 366, 837, 257, 23634, 2422, 4326, 7351, 262, 3277, 286, 7096, 544, 1141, 262, 5498, 1898, 6839, 1810, 508, 1620, 3200, 2042, 4560, 290, 389, 46852, 1028, 262, 11773, 4326, 366, 2199, 321, 265, 88, 12552, 366, 764, 220], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
t['input_ids'] += t['knowledge']['input_ids']
t['attention_mask'] += t['knowledge']['attention_mask']
t

{'input_ids': [2311, 73, 13090, 645, 569, 18354, 7496, 513, 1058, 791, 47398, 17740, 357, 4960, 1058, 10545, 230, 99, 161, 254, 112, 5641, 44444, 9202, 25084, 24440, 12675, 11839, 18, 837, 6578, 764, 569, 18354, 7496, 286, 262, 30193, 513, 1267, 837, 8811, 6412, 284, 355, 569, 18354, 7496, 17740, 6711, 2354, 2869, 837, 318, 257, 16106, 2597, 2488, 12, 31, 2712, 2008, 983, 4166, 416, 29490, 290, 6343, 13, 44206, 329, 262, 14047, 44685, 764, 28728, 287, 3269, 2813, 287, 2869, 837, 340, 318, 262, 2368, 983, 287, 262, 569, 18354, 7496, 2168, 764, 12645, 278, 262, 976, 21748, 286, 16106, 290, 1103, 2488, 12, 31, 640, 11327, 355, 663, 27677, 837, 262, 1621, 4539, 10730, 284, 262, 717, 983, 290, 5679, 262, 366, 17871, 5321, 366, 837, 257, 23634, 2422, 4326, 7351, 262, 3277, 286, 7096, 544, 1141, 262, 5498, 1898, 6839, 1810, 508, 1620, 3200, 2042, 4560, 290, 389, 46852, 1028, 262, 11773, 4326, 366, 2199, 321, 265, 88, 12552, 366, 764, 220, 4895, 53, 18354, 7496, 17740, 6711, 1298, 19779, 11213

In [ ]:
t['input_ids'] = torch.LongTensor(t['input_ids'])
t['attention_mask'] = torch.LongTensor(t['attention_mask'])
t['labels'] = torch.LongTensor(t['labels'])
t

{'input_ids': tensor([ 2311,    73, 13090,   645,   569, 18354,  7496,   513,  1058,   791,
        47398, 17740,   357,  4960,  1058, 10545,   230,    99,   161,   254,
          112,  5641, 44444,  9202, 25084, 24440, 12675, 11839,    18,   837,
         6578,   764,   569, 18354,  7496,   286,   262, 30193,   513,  1267,
          837,  8811,  6412,   284,   355,   569, 18354,  7496, 17740,  6711,
         2354,  2869,   837,   318,   257, 16106,  2597,  2488,    12,    31,
         2712,  2008,   983,  4166,   416, 29490,   290,  6343,    13, 44206,
          329,   262, 14047, 44685,   764, 28728,   287,  3269,  2813,   287,
         2869,   837,   340,   318,   262,  2368,   983,   287,   262,   569,
        18354,  7496,  2168,   764, 12645,   278,   262,   976, 21748,   286,
        16106,   290,  1103,  2488,    12,    31,   640, 11327,   355,   663,
        27677,   837,   262,  1621,  4539, 10730,   284,   262,   717,   983,
          290,  5679,   262,   366, 17871,  5321, 

In [ ]:
model.forward(t)

> /home/rob/miniconda3/envs/kirby/lib/python3.8/site-packages/transformers/models/gpt2/modeling_gpt2.py(969)forward()
    967             # Shift so that tokens < n predict n
    968             import pdb; pdb.set_trace()
--> 969             shift_logits = lm_logits[..., :-1, :].contiguous()
    970             shift_labels = labels[..., 1:].contiguous()
    971             # Flatten the tokens



ipdb>  n


> /home/rob/miniconda3/envs/kirby/lib/python3.8/site-packages/transformers/models/gpt2/modeling_gpt2.py(970)forward()
    968             import pdb; pdb.set_trace()
    969             shift_logits = lm_logits[..., :-1, :].contiguous()
--> 970             shift_labels = labels[..., 1:].contiguous()
    971             # Flatten the tokens
    972             loss_fct = CrossEntropyLoss()



ipdb>  n


> /home/rob/miniconda3/envs/kirby/lib/python3.8/site-packages/transformers/models/gpt2/modeling_gpt2.py(972)forward()
    970             shift_labels = labels[..., 1:].contiguous()
    971             # Flatten the tokens
--> 972             loss_fct = CrossEntropyLoss()
    973             loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    974 



ipdb>  shift_logits.shape


torch.Size([180, 50257])


ipdb>  shift_labels.shape


torch.Size([180])
